# Going Subphraseless

The current method for isolating phrase heads ([here](https://nbviewer.jupyter.org/github/ETCBC/heads/blob/master/phrase_heads.ipynb)) requires strenuous and ineloquent processing of BHSA subphrase relations. The subphrases are not always consistently encoded and suffer from numerous exceptional cases. The result is that the method is rather convoluted and ineloquent.

This notebook will explore the possibility of disconnecting semantic head analysis from the ETCBC subphrase encoding. 

A "semantic" head is the primary content word of a phrase, following Croft's "Primary Information Bearing Unit":

> **The noun and the verb are the PRIMARY INFORMATION_BEARING UNITS (PIBUs) of the phrase and clause respectively. In common parlance, they are the content words. PIBUs have major informational content that functional elements such as articles and [auxiliaries] do not have. (Croft, *Radical Construction Grammar*, 2001, 258; see also Shead, *Radical Frame Semantics and Biblical Hebrew*, 104)**

> **A (semantic) head is the profile equivalent that is the primary information-bearing unit, that is, the most contentful item that most closely profiles the same kind of thing that the whole constituent profiles. (ibid., 259)**

Croft also provides an additional criterion to "profile equivalence":

> **If the criterion of profile equivalence produces two candidates for headhood, the less schematic meaning is the PIBU; that is, the PIBU is the one with the narrower extension, in the formal semantic sense of that term (ibid., 259)**

## Inquiry

Can we isolate semantic phrase heads in BHSA using only the phrase_atom and phrase limits? This question indeed means that we  take the phrase_atom/phrase boundaries for granted. Empirically, the validity of BHSA phrase boundaries needs to be tested. But for now, the exercise of isolating semantic phrase heads could be seen as the first step towards reproducible phrase boundaries.

## Basic Concepts

A semantic head will most often stand in a syntactically independent position. For Hebrew nominal phrases, that essentially means a word which is not precided by a construct, and which is semantically central (excluding attributive slots (e.g. H + noun + H + ATTRIBUTIVE) or an adjectival slots (e.g. noun + noun as in אישׁ טוב).

Quantifier expressions present unique cases, which may be syntactically independent but semantically secondary. These are expressed through specialized lexical items such as cardinal numbers and qualitative quantifiers (e.g.  "כל" and "חצי").

Another complication is the use of nouns as prepositional items. Such uses can be seen with words like פני "face" such as לפני "in front," and even words like ראשׁ as in ראשׁ החדשׁ "beginning of the month." 

Other expressions of quantity, quality, and function provide similar complexities. These cases have to be specified in advance.

### Ambiguity

Considerable ambiguity is present in several of cases:

**`A B and C`**<br>
Given A, B, C == nominal words. Is their relationship `A // B // C` or `A+B // C`. In other words: **what is the relationship of two adjacent nominal words given a list?** Is B a descriptor of A or is it an independent element? 

**`A of B and C`**<br>
Is it, `(A of B) // (C)` or `(A of (B // C)`

Or even:

**`A of B C and D`**<br>
This pattern combines elements from both ambiguous cases.

To address these ambiguities we will apply a battery of disambiguation attempts. Some of those attempts will draw from corpus data, i.e. do we ever see `B and C` with the conjunction explicitly elsewhere in the corpus? Or do we ever see a `A of C` excplicitly in the corpus? Accents may also play a role: do we see a conjunctive or disjunctive accent between `B C`? 

## Prerequisites

A number of pre-defined word sets are needed for processing quantification and ambiguous adjacency. These sets are made available in the form of `wsets`, a dictionary containing word sets that are calculated in to the `wordsets` directory of this repository. The following wordsets have been defined:

* nominals – a set of word nodes with parts of speech and participles that have the potential to function as nominalized elements. The selected parts of speech are quite permissive: `{'subs', 'nmpr', 'adjv', 'advb', 'prde', 'prps', 'prin', 'inrg'}`. Since parts of speech are not taken as universal linguistic categories but only summaries of language-specific word tendencies (cf. Croft, *Radical Construction Grammar*, 2001), we consider that almost any part of speech can be used in a nominal pattern (or construction). There are some upper limits to this assumption, though. For instance, we exclude cojunctions, articles, prepositions, and negators. 
* prepositions – a word set consisting of words with a part of speech category of `prep`, a lexical set (`ls`) feature of `ppre` ("potential preposition"), as well as a select group of nouns like פני "face" which have been processed for prepositionality. 
* quantifiers - consists of word nodes that are cardinal numbers or qualitative quantifiers such as כל.
* mword – mapping from a word to its phonological word group ("masoretic word"); joins words on maqqeph and ø space
* accent_type – a mapping from a word to its accent type: conjunctive or disjunctive
* conj_pairs – a dict of observed conjunction pairings of lexemes in the corpus: `A & B`
* cons_pairs – a dict of observed construct pairings of lexemes in the corpus: `A of B`
* mom – mapping from word node to its mother word node for a specified relationship: `mom[A]['coord'] = B`
* kid – opposite of mom; mapping from word to its children nodes for a relationship: `kid[A]['cons'] = B`

**Let's get started**. We load the necessary functions and BHSA data (straight from source).

In [1]:
import collections
import pickle
import random
import re
from IPython.display import display, HTML
from datetime import datetime
from pprint import pprint
from tf.app import use
wsets = pickle.load(open('wordsets/wsets.pickle', 'rb'))
A = use('bhsa', hoist=globals(), silent=True)

   |     0.00s No structure info in otext, the structure part of the T-API cannot be used


### Wordsets

In [2]:
wsets.keys()

dict_keys(['noms', 'preps', 'quants', 'accent_type', 'mwords', 'conj_pairs', 'cons_pairs'])

In [3]:
list(wsets['cons_pairs'].keys())[:10]

['>JC/', 'KL/', 'BN/', 'TPF[', 'MZBX/', 'B>R/', '<T/', 'XRB/', 'RWX/', 'MLK/']

In [4]:
#wsets['conj_pairs']['>JC/']

# Machinery

We could use some machinery to do the hard work of looking in and around a node. In the older approach we used TF search templates. But these are not very efficient at scale, and they are always bound by the limits of the query language. I take another approach here: a set of classes that specify locations and directions within a specified context.

In [5]:
from wordsets.positions import Positions, Walker

## Positions class

The `Positions` class enables concise access to adjacent nodes within a given context. This allows us to write algorithms with query-like efficiency with all of the power of Python. 

This class is instantiated on a word node and can provide contextual look-up data for a given word. For example, given a phrase containing the following word nodes:

> (189681, 189682, **189683**, 189684, 189685, 189686) <br>

representing the following phrase (space separated for clarity):

> ב שׁנת **שׁלשׁים** ו שׁמנה שׁנה

Given that the bolded node, `189683` is our `source` word, we instantiate the class, feeding in the node, the "phrase_atom" string (which is the context we want to search within), and an instance of Text-Fabric (`tf`):

In [6]:
      #    source node    context  TF instance  
      #         |            |       |
P = Positions(189683, 'phrase_atom', A).get

If we want to obtain the word adjacent one space forward, we simply ask `P` for `1`, which gives us the next word in the phrase.

In [7]:
P(1)

189684

If we try to ask for 4 words forward, we go beyond the bounds of the phrase. But `P` handles this by returning nothing:

In [8]:
P(4)

To look back one word, we simply give a negative value:

In [9]:
P(-1)

189682

Finally, `P` can be used to quickly call features on these words. For instance, in order to get the lexeme of the word two words in front of `189683`:

In [10]:
P(2,'lex')

'CMNH/'

And if we want to get a number of features, we can just add them with space-separation to the string. The result is a feature set:

In [11]:
P(2,'lex nu')

{'CMNH/', 'sg'}

`P` can also handle features on the source node itself by giving a positionality of `0`:

In [12]:
P(0, 'lex')

'CLC/'

## Walker class

`Walker` performs a similar function to `Positions`, except it is ambiguous to exact positions, walking either `ahead` or `back` from the source to a target node in the context. A function must be supplied that returns `True` on the target node.

We instantiate the `Walker` using the same source and context as above.

In [13]:
      #  source node    context  TF instance  
      #       |            |       |
Wk = Walker(189683, 'phrase_atom', A)

`Walker` is demonstrated below with the same word. A simple `lambda` function is used to test for the lexeme. In the example below, we find the first word ahead of `189683` that is a cardinal number:

In [14]:
Wk.ahead(lambda w: F.ls.v(w) == 'card')

189685

An alternative demonstrates the `None` returned on the lack of a valid match.

In [15]:
Wk.ahead(lambda w: F.ls.v(w) == 'BOOGABOOGA')

Another example wherein we walk backwards to the preposition:

In [16]:
Wk.back(lambda w: F.sp.v(w) == 'prep')

189681

We can also specify that the walk should be interrupted under certain conditions with a `stop` function. In this case we walk forward to the next cardinal number, but the walk is interrupted when the `stop` function detects a conjunction.

In [17]:
Wk.ahead(lambda w: F.ls.v(w) == 'card',
         stop=lambda w: F.sp.v(w) == 'conj')

We can also specify the opposite with a `go` function argument, which defines the nodes that allowed to intervene between `source` and `target`. Below we specify that *only* a conjunction should intervene.

In [18]:
Wk.ahead(lambda w: F.ls.v(w) == 'card',
         go=lambda w: F.sp.v(w) == 'conj')

189685

The `go` and `stop` functions can be as permissive or strict as desired.

# Need for Semantic Data

The accurate processing of word connections depends on fuller semantic data than BHSA provides. Future semantic data could be stored in a similar way to word sets (`wsets`). 

For example, in the two phrases

> (Exod 25:39) ככר זהב טהור <br>
> (2 Sam 24:24) בכסף שקלים חמשׁים

we see that זהב and כסף, despite being in two different positions with two different words indicates a kind of "composed of" semantic concept: "round gold" (i.e. round composed of gold) and "silver shekels" (shekels composed of silver). To process these kinds of links, we need a list of nouns that often function as "material." But this is only the beginning. Many other words will have specific semantic values that motivate their syntactic behavior. Such a scope lies outside the bounds of this author's current project on Hebrew time phrases.

## A Compromise: Time Phrases

Since constructing these semantic classes is vastly time consuming, I want to start with a smaller set of cases. I will instead focus on parsing connections within time phrases for now. This is because I am analyzing time phrases in my current ongoing PhD project. 

In [19]:
timephrases = [ph for ph in F.otype.s('phrase_atom') 
                   if F.function.v(L.u(ph, 'phrase')[0]) == 'Time'
                   and len(L.d(ph, 'word')) > 1
                   and F.language.v(L.d(ph, 'word')[0]) == 'Hebrew'
            ]

print(f'{len(timephrases)} phrases ready')

3766 phrases ready


## Search & Display Functions

The functions below allow for fast searching and displaying of queries using the `Positions` class. The searches rely on the `Grammar` class, described further below.

In [20]:
def prettyconds(rulesets):
    '''
    Iterate through an explain dict for a rela
    and print out all of checked conditions.
    '''
    
    for ruleset in rulesets:
        name, src, tgt = ruleset['name'], ruleset['src'], ruleset['tgt']
        print(name)
        print(f'{src} -> {tgt}')
        
        for cond, value in ruleset['cnd'].items():
            print('{:<30} {:>30}'.format(cond, str(value)))
        
        print()
        
def showmatch(match):
    
    '''
    Displays a match from a Grammar test.
    '''
    
    hit, conds = match['match'], match['conds']
    
    if not hit:
        print('NO MATCHES')
        print('-'*20)
        A.pretty(L.u(conds[0]['src'], 'phrase_atom')[0], extraFeatures='sp st', withNodes=True)
        prettyconds(conds)
        return None

    name, src, tgt = hit['name'], hit['src'], hit['tgt']

    phrase = L.u(src, 'phrase_atom')[0]

    highlights = {src:'pink',
                  tgt:'lightgreen'}

    A.pretty(phrase, withNodes=True, extraFeatures='sp st', highlights=highlights)
    prettyconds(conds)
    display(HTML('<hr>'))
        
def test_search(relastr, show=10, end=None, name='', phrases=None):
    '''
    Searches phrases with the specified relation 
    and prints out their descriptive explanation.
    '''
    
    start = datetime.now()
    print('beginning search')
    
    # build a convenient test set of words
    phrases = phrases or [ph for ph in F.otype.s('phrase_atom') 
                   if F.typ.v(ph) in {'NP', 'PP'}
                   and len(L.d(ph, 'word')) > 1
              ]
    words = [w for ph in phrases for w in L.d(ph, 'word')]
    
    # random shuffle to get good diversity of examples
    random.shuffle(words)
    
    # set up grammar
    G = NounGrammar(wsets, A)
    
    matches = []
    append = matches.append
    
    # iterate and find matches on words
    for i,w in enumerate(words):

        # update every 5000 iterations
        if i%5000 == 0:
            print(f'\t{len(matches)} found ({i}/{len(words)})')
        
        # run grammar search
        test = G.tests[relastr](w)
        
        # save results
        if test['match']:
            if not name:
                append(test)
            elif test['match']['name'] == name:
                append(test)
            
        # stop at end
        if len(matches) == end:
            break
        
        
    # display
    print('done at', datetime.now() - start)
    print(len(matches), 'matches found...')
    print('showing', end)
    
    for match in matches[:show]:
        showmatch(match)

## NounGrammar class

While `Positions` provides concise access to context, `Grammar` classes contain a series of functions which test a bunch of conditions. The conditions are formed by testing word `Positions`. An example of the `Grammar` class is provided below.

In [21]:
class NounGrammar:
    
    def __init__(self, wsets, tf):
        self.tf = tf
        self.wsets = wsets
        self.tests = {
            'defi': self.defi,
            'prep': self.prep,
            'geni': self.geni,
            'adjv': self.adjv,
            'quant': self.quant,
            'card_chain': self.card_chain
         }
        # cache results for recursive calls?
        self.cache = collections.defaultdict(lambda:collections.defaultdict())
        
    def getP(self, w):
        return Positions(w, 'phrase_atom', self.tf).get
    
    def getWk(self, w):
        return Walker(w, 'phrase_atom', self.tf)
    
    def test(self, condtuple):
        test = [ruleset for ruleset in condtuple
                    if all(ruleset['cnd'].values())
               ]
        
        # NB rules are written so as to default
        # on the last-most matching item
        # this allows for more complex situations
        # to default if true
        if test:
            return test[-1]
        else:
            return {}
        
    def everymatch(self, w):
        '''
        Runs analysis for all tests on a word.
        Returns as dict of test:result.
        '''
        results = {}
        
        for name, test in self.tests.items():
            res = test(w)
            if res['match']:
                results[name] = res['match']
                
        return results
        
    def defi(self, w):
        '''
        Matches a definite article with noun.
        '''
        
        params = {'match': {}, 'conds':None}
        
        if not w:
            return params
                
        P = self.getP(w)
        
        params['conds'] = (
        
        {
            'name': 'definite',
            'src': P(0),
            'tgt': P(1),
            'cnd': {
                
                'F.sp.v(w) == art':
                    F.sp.v(w) == 'art',
                
                'bool(P(1))':
                    bool(P(1))
            }
            
        },
        )
        
        params['match'] = self.test(params['conds'])
        return params
    
    def prep(self, w):
        '''
        Matches a prepositional object.
        '''
        
        params = {'match': {}, 'conds':None}
        
        if not w:
            return params
                
        P = self.getP(w)
        
        params['conds'] = (
        
            {
                'name': 'prep',
                'src': w,
                'tgt': P(1),
                'cnd': {

                    'w in preps':
                        w in self.wsets['preps'],

                    'F.prs.v(w) == absent':
                        F.prs.v(w) == 'absent',
                    
                    'bool(P(1))':
                        bool(P(1)),
                }
            },
        )
        
        params['match'] = self.test(params['conds'])
        return params
        
    def geni(self, w):
        
        '''
        Queries for "genitive" relations on a word.
        '''
        
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, preps, noms = wsets['quants'], wsets['preps'], wsets['noms']
        
        params['conds'] = (

            {
                'name': 'geni',
                'src': P(0),
                'tgt': P(-1),
                'cnd': {

                    'P(-1, st) == c': 
                        P(-1,'st') == 'c',

                    'P(-1) not in quants|preps': 
                        P(-1) not in quants|preps,
                }
            },
        )

        params['match'] = self.test(params['conds'])
        return params

    def adjv(self, w):
        
        '''
        Searches for adjective relas on a word.
        '''
           
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, preps, noms = wsets['quants'], wsets['preps'], wsets['noms']
        
        # check for recursive adjective matches 
        a2match = self.adjv(P(-1))
        
        common = {
            
            'P(-1) in noms':
                P(-1) in noms,
            
            'P(-1, st) & {NA, a}': 
                P(-1,'st') in {'NA', 'a'},   
            
            'P(-1) not in {quants|preps}':
                P(-1) not in quants|preps,
        }
                
        params['conds'] = (
            
            {
                'name': 'adjv',
                'src': P(0),
                'tgt': P(-1),
                'cnd': dict(common, **{

                    'P(0, sp) in {adjv, verb}':
                        P(0,'sp') in {'adjv', 'verb'},

                })
            },
            
            {
                'name': 'adjv adjv',
                'src': P(0),
                'tgt': a2match.get('tgt', None),
                'cnd': dict(common, **{
                    
                    'P(0,sp) in {adjv, verb}':
                        P(0,'sp') in {'adjv', 'verb'},
                    
                     'self.adjv(P(-1)) and target != P(0)':
                        bool(a2match['match']) and a2match.get('tgt', None) != P(0)
                })
            },
            
           {
                'name': 'adverb',
                'src': P(0),
                'tgt': P(1),
                'cnd': {

                    'P(0,sp) == advb':
                        P(0,'sp') == 'advb',

                    'P(1) in noms':
                        P(1) in noms,
                }
            },
            
            {
                'name': 'attrib pattern',
                'src': P(0),
                'tgt': P(-2),
                'cnd': {
                    
                    'P(0,sp) not in {prep, art, conj}':
                        P(0,'sp') not in {'prep', 'art', 'conj'},
                    
                    'P(0) not in preps':
                        P(0) not in preps,
                    
                    'P(-1,sp) == art':
                        P(-1,'sp') == 'art',
                    
                    'P(-2) in noms':
                        P(-2) in noms,
                    
                    'P(-2) not in quants':
                        P(-2) not in quants,
                    
                    'P(-2,st) in {NA, a}':
                        P(-2,'st') in {'NA', 'a'},
                    
                    'P(-2,sp) != advb':
                        P(-2,'sp') != 'advb',
                    
                }
                
            }
        )

        params['match'] = self.test(params['conds'])
        return params
     
        
    def quant(self, w):
        """Defines quantification with an non-quant word"""
        
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        Wk = self.getWk(w)
        is_nom = (lambda n: n in self.wsets['noms'] 
                      and n not in self.wsets['quants'])
        behind_nom = Wk.back(is_nom, go=lambda n: F.sp.v(n)=='art') 

        params['conds'] = (
        
            {
                'name': 'quant forward',
                'src': w,
                'tgt': P(1),
                'cnd': {
                    'w in quants':
                        w in self.wsets['quants'],
                    
                    'P(1,sp) != conj':
                       P(1,'sp') != 'conj',
                    
                    'P(1) not in quants':
                        P(1) not in self.wsets['quants'],
                    
                    'bool(P(1))':
                        bool(P(1))
                },
            },  
            {
                'name': 'quant backward',
                'src': w,
                'tgt': behind_nom,
                'cnd': {
                    
                    'w in quants':
                        w in self.wsets['quants'],
                    
                    'not Wk.ahead(is_nominal)':
                        not Wk.ahead(is_nom),
                    
                    'bool(Wk.back(is_nominal))':
                        bool(behind_nom)
                }
            }
        )
        
        params['match'] = self.test(params['conds'])
        return params
        
    def card_chain(self, w):
        """Defines cardinal number conjunctions."""
        
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        Wk = self.getWk(w)
        is_card = (lambda n: F.ls.v(n) == 'card')
        back_card = Wk.back(is_card, go=lambda n: F.sp.v(n)=='conj')
        
        params['conds'] = (
        
            {
                'name': 'card connective',
                'src': w,
                'tgt': back_card,
                'cnd': {
                    
                    'F.ls.v(w) == card':
                        F.ls.v(w) == 'card',
                    
                    'bool(Wk.back(is_card), go=conj)':
                        bool(back_card),                    
                }
            },
        )
        
        params['match'] = self.test(params['conds'])
        return params

In [22]:
#     def quant(self, w):
        
#         params = {'match': {}, 'conds': None}
        
#         if not w:
#             return params
        
#         P = self.getP(w)
#         wsets = self.wsets
#         quants, noms = wsets['quants'], wsets['noms']
    
#         get_quanted = self.get_quantified(w)
        
#         qqbackward = self.quant(P(-1))
        
#         common = {
#             'P(0) in quants':
#                 P(0) in quants,
#         }
        
#         params['conds'] = (
        
#             {
#                 'name': 'quant forward',
#                 'src': P(0),
#                 'tgt': get_quanted,
#                 'cnd': dict(common, **{
                    
#                     'bool(get_quanted)':
#                         bool(get_quanted)
#                 })
#             },
            
#             {
#                 'name': 'quant backward',
#                 'src': P(0),
#                 'tgt': P(-1),
#                 'cnd': dict(common, **{
                
#                     'not bool(get_quanted)':
#                         not bool(get_quanted),
                    
#                     'P(-1,st) in {NA, a}':
#                         P(-1,'st') in {'NA', 'a'},
                    
#                     'P(-1) in noms':
#                         P(-1) in noms,
                    
#                     'P(-1) not in quants':
#                         P(-1) not in quants
                    
#                 })
#             },
            
#             {
#                 'name': 'quant backward w/ article',
#                 'src': P(0),
#                 'tgt': P(-2),
#                 'cnd': dict(common, **{
                
#                     'not bool(get_quanted)':
#                         not bool(get_quanted),
                    
#                     'P(-1, sp) == art':
#                         P(-1,'sp') == 'art',
                    
#                     'P(-2,st) in {NA, a}':
#                         P(-2,'st') in {'NA', 'a'},
                    
#                     'P(-2) in noms':
#                         P(-2) in noms,
                    
#                     'P(-1) not in quants':
#                         P(-2) not in quants
                    
#                 })
#             },
        
#             {
#                 'name': 'quant quant backward',
#                 'src': P(0),
#                 'tgt': qqbackward['match'].get('tgt', None),
#                 'cnd': dict(common, **{
                
#                     'not bool(get_quanted)':
#                         not bool(get_quanted),
                    
#                     'bool(qqbackward[match])':
#                         bool(qqbackward['match']),
#                 })
#             },
            
        
#         )
        
#         params['match'] = self.test(params['conds'])
#         return params
    
#     def get_quantified(self, word):
#         '''
#         Locates the first non-quant nominal word
#         ahead of a quantifier.
#         '''

#         if not word:
#             return None
        
#         quants, noms = self.wsets['quants'], self.wsets['noms']
        
#         P = G.getP(word)

#         conds = (
#             P(0), 
#             P(0) not in quants,
#             P(0) in noms,
#         )

#         # check this word
#         if all(conds):
#             return word

#         # move up one
#         else:
#             return self.get_quantified(P(1))

In [23]:
G = NounGrammar(wsets, A)

In [24]:
wsets['accent_type'][76403]

'disjunct'

In [25]:
showmatch(G.quant(4018))

NO MATCHES
--------------------


quant forward
4018 -> None
w in quants                                             False
P(1,sp) != conj                                          True
P(1) not in quants                                       True
bool(P(1))                                              False

quant backward
4018 -> 4016
w in quants                                             False
not Wk.ahead(is_nominal)                                 True
bool(Wk.back(is_nominal))                                True



In [26]:
#showmatch(G.quant(62184))

## Testing

In [27]:
#test_search('cons', name='', show=100, end=50, phrases=timephrases)

## Generate Chunks

For every word in the timephrase set that possesses a dependent relation, generate a chunk object with that word as its head. Also, generate prepositional chunks.

In [28]:
relas = {}

nomless = []

# set up Grammar class
G = NounGrammar(wsets, A)

# time it
start = datetime.now()

print(f'{datetime.now()-start} beginning analysis...')

for i, phrase in enumerate(timephrases):
        
    # analyze all known relas
    for w in L.d(phrase, 'word'):
        analysis = G.everymatch(w)
        if analysis:
            relas[w] = analysis
        
    # report status
    if i % 500 == 0 and i:
        print(f'\t{datetime.now()-start}\tdone with iter {i}/{len(timephrases)}')
        
print(f'{datetime.now()-start}\tCOMPLETE')

0:00:00.000035 beginning analysis...
	0:00:15.817151	done with iter 500/3766
	0:00:31.263590	done with iter 1000/3766
	0:00:49.097439	done with iter 1500/3766
	0:01:02.668819	done with iter 2000/3766
	0:01:18.709811	done with iter 2500/3766
	0:01:32.463464	done with iter 3000/3766
	0:01:46.612996	done with iter 3500/3766
0:01:54.822091	COMPLETE


### Compose Relations 

In [29]:
kid2mom = collections.defaultdict(list)

for word, rels in relas.items():
    for rela, data in rels.items():
        kid2mom[word].append((rela, data['tgt']))
        
mom2kids = collections.defaultdict(list)

for kid, rels in kid2mom.items():
    for rela, mom in rels:
        mom2kids[mom].append((rela, kid))
        
# change to regular dict to prevent accidental insertions during testing
kid2mom = dict(kid2mom)
mom2kids = dict(mom2kids)

In [ ]:
kid2mom = collections.defaultdict(list)

for word, rels in relas.items():
    for rela, data in rels.items():
        kid2mom[word].append((rela, data['tgt']))
        
mom2kids = collections.defaultdict(list)

for kid, rels in kid2mom.items():
    for rela, mom in rels:
        mom2kids[mom].append((rela, kid))
        
kid2mom = dict(kid2mom)
mom2kids = dict(mom2kids)

In [35]:
kid2mom[189685]

[('quant', 189686), ('card_chain', 189683)]

In [61]:
testph = 907060

head2tree = {}

def findmom(w, relastr):
    """Find specified mom in a tree."""
    for rel,node in kid2mom[w]:
        if rel == relastr:
            return node
        
def findkid(w, relastr):
    """Find specified kid in a tree."""
    for rel,node in mom2kids[w]:
        if rel == relastr:
            return node
    
def findtree(mom, covered, dolinks=True):
    
    # link together dependent chains
    dep_links = {
        'card_chain': {
            'link':'card_chain',
            'tgt': 'quant'
        },
    }
    
    covered.add(mom)
        
    tree = [('head', [mom])] 
        
    test = 189683
        
    for rela, kid in mom2kids[mom]:
        
        # handle cases where a fronted element is dependent
        # on a mother connected via a chain of elements it governs
        if rela in dep_links and dolinks:
            
            link = dep_links[rela]['link']
            target = dep_links[rela]['tgt']            
            thisnode = kid
            
            # iterate down through links until finding target
            while thisnode:
                
                # stop if target found and do tree from target
                get_tgt = findmom(kid, target)
                if get_tgt:
                    covered.add(get_tgt)
                    tree = [('head', [get_tgt]), ('quant', findtree(mom, covered, dolinks=False))]
                    break
                
                # continue if link continued
                thisnode = findkid(thisnode, link)
                
            # stop at this kid if search was success
            if thisnode:
                continue
        
        # store simple kid/relas
        if len(kid2mom[kid]) == 1 and kid not in mom2kids:
            tree.append((rela, [kid]))
            
        # retrieve kid's own tree
        elif kid in mom2kids:                
            kidtree = findtree(kid, covered)
            tree.append((rela, kidtree))
            
        # retrieve kid's mom's tree for elements not yet covered
        else:

            
            for rela2, mom2 in kid2mom[kid]:
                if mom2 != mom and mom2 not in covered:
                    dephrase = findtree(mom2, covered)
                    tree.append((rela, dephrase))
                    
                elif mom2 != mom:
                    tree.append((rela, [kid]))
            
    return tree
                    
                                                        
                
# for w in L.d(testph, 'word'):
#     if w in mom2kids:
#         findtree(w)

In [81]:
covered = set()

for w in L.d(760754, 'word'):
    
    if w in covered:
        continue
    
    if w in mom2kids:
    
        print(findtree(w, covered))    

[('head', [189682]), ('prep', [189681]), ('geni', [('head', [189686]), ('quant', [('head', [189683]), ('card_chain', [189685])])])]
[('head', [189688]), ('prep', [189687])]
[('head', [189689]), ('geni', [189690])]


In [80]:
A.pretty(760754, withNodes=True, extraFeatures='st')

In [75]:
# A.show(A.search('''

# phrase function=Time
#     word ls=card
#     <: word ls=card
#     <1: word ls=card

# ''')[:20], withNodes=True)

In [35]:
A.pretty(907060, withNodes=True)

In [36]:
covered = set()

for w in L.d(907060, 'word'):
    
    if w in covered:
        continue
    
    if w in mom2kids:
        
        print(findtree(w, covered))
        

[('head', [4016]), ('quant', [4015]), ('geni', [('head', [4018]), ('defi', [4017])])]


### Compose Complex Chunks

In [155]:
multiwords = []

for w, matches in relas.items():
    
    moms = set()
    
    for rela, data in matches.items():
        moms.add(data['tgt'])
    
    if len(moms) > 1:
        multiwords.append(w)
        
len(multiwords)

145

In [163]:
relas[382387]

{'defi': {'name': 'definite',
  'src': 382387,
  'tgt': 382388,
  'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}},
 'cons': {'name': 'cons',
  'src': 382387,
  'tgt': 382386,
  'cnd': {'P(-1, st) == c': True, 'P(-1) not in quants|preps': True}}}

In [164]:
# for w in multiwords:
#     print(w, T.text(w))

In [109]:
# TO TEST:
# 760754 
# see how to propogate relas upwards
# e.g. preps, or constructs

In [86]:
shuff = [ph for ph in timephrases 
             if ph not in nomless
            and len(L.d(ph, 'word')) > 5
        ]
random.shuffle(shuff)
    
for phrase in shuff[:100]:

    A.pretty(phrase, withNodes=True)
    
    for w in L.d(phrase , 'word'):
        
        print(T.text(w), w)

        for rela, match in relas.get(w, {}).items():

            print(f'\t** {rela} ** ')
            print('\t', match)
            print()

        print('-'*20)
        
    print()
    display(HTML('<hr>'))

בִּ 189681
--------------------
שְׁנַ֨ת  189682
--------------------
שְׁלֹשִׁ֤ים  189683
	** cons ** 
	 {'name': 'cons', 'src': 189683, 'tgt': 189682, 'cnd': {'P(-1, st) == c': True, 'P(-1) not in quants|preps': True}}

	** quant ** 
	 {'name': 'quant forward', 'src': 189683, 'tgt': 189686, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
וּ 189684
--------------------
שְׁמֹנֶה֙  189685
	** quant ** 
	 {'name': 'quant forward', 'src': 189685, 'tgt': 189686, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
שָׁנָ֔ה  189686
--------------------



כָּל־ 153391
	** quant ** 
	 {'name': 'quant forward', 'src': 153391, 'tgt': 153393, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
הַ 153392
	** defi ** 
	 {'name': 'definite', 'src': 153392, 'tgt': 153393, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
יֹּ֥ום  153393
--------------------
הַ 153394
	** defi ** 
	 {'name': 'definite', 'src': 153394, 'tgt': 153395, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
ה֖וּא  153395
	** adjv ** 
	 {'name': 'attrib pattern', 'src': 153395, 'tgt': 153393, 'cnd': {'P(0,sp) not in {prep, art, conj}': True, 'P(0) not in preps': True, 'P(-1,sp) == art': True, 'P(-2) in noms': True, 'P(-2) not in quants': True, 'P(-2,st) in {NA, a}': True, 'P(-2,sp) != advb': True}}

--------------------
וְ 153396
--------------------
כָל־ 153397
	** quant ** 
	 {'name': 'quant forward', 'src': 153397, 'tgt': 153399, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}

חָמֵ֤שׁ  2556
	** quant ** 
	 {'name': 'quant forward', 'src': 2556, 'tgt': 2559, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
וְ 2557
--------------------
תִשְׁעִים֙  2558
	** quant ** 
	 {'name': 'quant forward', 'src': 2558, 'tgt': 2559, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
שָׁנָ֔ה  2559
--------------------
וַ 2560
--------------------
חֲמֵ֥שׁ  2561
	** quant ** 
	 {'name': 'quant forward', 'src': 2561, 'tgt': 2563, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
מֵאֹ֖ת  2562
	** quant ** 
	 {'name': 'quant forward', 'src': 2562, 'tgt': 2563, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
שָׁנָ֑ה  2563
--------------------



רַ֚ק  115711
--------------------
בַּ 115712
--------------------
 115713
	** defi ** 
	 {'name': 'definite', 'src': 115713, 'tgt': 115714, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
יֹּ֣ום  115714
--------------------
הַ 115715
	** defi ** 
	 {'name': 'definite', 'src': 115715, 'tgt': 115716, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
ה֔וּא  115716
	** adjv ** 
	 {'name': 'attrib pattern', 'src': 115716, 'tgt': 115714, 'cnd': {'P(0,sp) not in {prep, art, conj}': True, 'P(0) not in preps': True, 'P(-1,sp) == art': True, 'P(-2) in noms': True, 'P(-2) not in quants': True, 'P(-2,st) in {NA, a}': True, 'P(-2,sp) != advb': True}}

--------------------



שְׁתַּ֧יִם  2371
	** quant ** 
	 {'name': 'quant forward', 'src': 2371, 'tgt': 2374, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
וְ 2372
--------------------
שִׁשִּׁ֛ים  2373
	** quant ** 
	 {'name': 'quant forward', 'src': 2373, 'tgt': 2374, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
שָׁנָ֖ה  2374
--------------------
וּ 2375
--------------------
מְאַ֣ת  2376
	** quant ** 
	 {'name': 'quant forward', 'src': 2376, 'tgt': 2377, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
שָׁנָ֑ה  2377
--------------------



...

לַ 66407
--------------------
 66408
	** defi ** 
	 {'name': 'definite', 'src': 66408, 'tgt': 66409, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
חֹדֶשׁ֩  66409
--------------------
הַ 66410
	** defi ** 
	 {'name': 'definite', 'src': 66410, 'tgt': 66411, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
שְּׁבִיעִ֨י  66411
	** adjv ** 
	 {'name': 'attrib pattern', 'src': 66411, 'tgt': 66409, 'cnd': {'P(0,sp) not in {prep, art, conj}': True, 'P(0) not in preps': True, 'P(-1,sp) == art': True, 'P(-2) in noms': True, 'P(-2) not in quants': True, 'P(-2,st) in {NA, a}': True, 'P(-2,sp) != advb': True}}

--------------------
הַ 66412
	** defi ** 
	 {'name': 'definite', 'src': 66412, 'tgt': 66413, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
זֶּ֜ה  66413
	** adjv ** 
	 {'name': 'attrib pattern', 'src': 66413, 'tgt': 66411, 'cnd': {'P(0,sp) not in {prep, art, conj}': True, 'P(0) not in preps': True, 'P(-1

בִּ 189357
--------------------
שְׁנַת֩  189358
--------------------
עֶשְׂרִ֨ים  189359
	** cons ** 
	 {'name': 'cons', 'src': 189359, 'tgt': 189358, 'cnd': {'P(-1, st) == c': True, 'P(-1) not in quants|preps': True}}

	** quant ** 
	 {'name': 'quant forward', 'src': 189359, 'tgt': 189362, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
וָ 189360
--------------------
שֶׁ֜בַע  189361
	** quant ** 
	 {'name': 'quant forward', 'src': 189361, 'tgt': 189362, 'cnd': {'P(0) in quants': True, 'bool(get_quanted)': True}}

--------------------
שָׁנָ֗ה  189362
--------------------



בַּ 49254
--------------------
 49255
	** defi ** 
	 {'name': 'definite', 'src': 49255, 'tgt': 49256, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
בֹּ֥קֶר  49256
--------------------
בַּ 49257
--------------------
 49258
	** defi ** 
	 {'name': 'definite', 'src': 49258, 'tgt': 49259, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
בֹּֽקֶר׃  49259
--------------------



בַּ 295003
--------------------
 295004
	** defi ** 
	 {'name': 'definite', 'src': 295004, 'tgt': 295005, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
יָּמִ֥ים  295005
--------------------
הָ 295006
	** defi ** 
	 {'name': 'definite', 'src': 295006, 'tgt': 295007, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
הֵ֖מָּה  295007
	** adjv ** 
	 {'name': 'attrib pattern', 'src': 295007, 'tgt': 295005, 'cnd': {'P(0,sp) not in {prep, art, conj}': True, 'P(0) not in preps': True, 'P(-1,sp) == art': True, 'P(-2) in noms': True, 'P(-2) not in quants': True, 'P(-2,st) in {NA, a}': True, 'P(-2,sp) != advb': True}}

--------------------
וּ 295008
--------------------
בָ 295009
--------------------
 295010
	** defi ** 
	 {'name': 'definite', 'src': 295010, 'tgt': 295011, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
עֵ֣ת  295011
--------------------
הַ 295012
	** defi ** 
	 {'name': 'definite', 'src': 295012

מִ 236533
--------------------
נְּעוּרֵ֖ינוּ  236534
--------------------
וְ 236535
--------------------
עַד־ 236536
--------------------
הַ 236537
	** defi ** 
	 {'name': 'definite', 'src': 236537, 'tgt': 236538, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
יֹּ֣ום  236538
--------------------
הַ 236539
	** defi ** 
	 {'name': 'definite', 'src': 236539, 'tgt': 236540, 'cnd': {'F.sp.v(w) == art': True, 'bool(P(1))': True}}

--------------------
זֶּ֑ה  236540
	** adjv ** 
	 {'name': 'attrib pattern', 'src': 236540, 'tgt': 236538, 'cnd': {'P(0,sp) not in {prep, art, conj}': True, 'P(0) not in preps': True, 'P(-1,sp) == art': True, 'P(-2) in noms': True, 'P(-2) not in quants': True, 'P(-2,st) in {NA, a}': True, 'P(-2,sp) != advb': True}}

--------------------



In [23]:
A.show(A.search('''

phrase function=Time
    word st=c lex#KL/|<YM/|>XRJT/ ls#card
    <: word lex=H
    <: word lex=JWM/

'''), end=20, condenseType='phrase', withNodes=True)

  2.24s 14 results


In [43]:
class PhraseGrammar:
    
    def __init__(self, ...):
    
    def disj(self, w):
        
        '''
        Queries for disjunct patterns, wherein A || B
        and B does not connect to A 
        A number of constructions are covered by this function:
            * lists - parallel items in a list
            * apposition - parallel items where B further specifies A
        '''
        
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, preps, noms = wsets['quants'], wsets['preps'], wsets['noms']
        conjpairs = wsets['conj_pairs']
        atype = wsets['accent_type']
        mword = wsets['mwords']
        
        # conditions common to several parameters
        common = {
            
           'P(0) in noms':
                P(0) in noms,

            'not {P(0), P(-1)} & quants':
                not {P(0), P(-1)} & quants,

            'P(-1) in noms':
                P(-1) in noms,

            'P(-1, st) == a': 
                P(-1,'st') == 'a',
            
        }
        
        # build the conditions
        params['conds'] = (

            {
                'name': 'disjoint conjpair',
                'src': P(0),
                'tgt': P(-1),
                'cnd': dict(common, **{
                                        
                    'P(-1,lex) in conjpairs[P(0,lex)]':
                        P(-1,'lex') in conjpairs[P(0,'lex')],
                })
            },
            
            {
                'name': 'disjoint accent',
                'src': P(0),
                'tgt': P(-1),
                'cnd': dict(common, **{
                    
                    'P(-1) == disjunct and P(-1) not in mword':
                        atype.get(P(-1)) == 'disjunct' and P(-1) not in mword[w],
                })
            },
        )

        params['match'] = self.test(params['conds'])
        return params
        
    def par(self, w):
        
        params = {'match': {}, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, noms, preps = wsets['quants'], wsets['noms'], wsets['preps']
        get_quanted = self.get_quantified(w)
        atype = wsets['accent_type']
        
        # run checks for disjoint, const, adjv
        disj = self.disj(w)
        cons = self.cons(w)
        adjv = self.adjv(w)
        quant = self.quant(P(-1))
        
        params['conds'] = (
        
            {
                'name': 'adjacent disjoint',
                'src': P(0),
                'tgt': disj['match'].get('tgt', None),
                'cnd': {
                    'disj match':
                        bool(disj['match'])
                }
            },
            
            {
                'name': 'adjacent, not const|adjv|quant',
                'src': P(0),
                'tgt': P(-1),
                'cnd': {
                    
                    'P(0) in noms':
                        P(0) in noms,
                    
                    'P(-1) in noms':
                        P(-1) in noms,
                    
                    'not cons match':
                        not bool(cons['match']),
                    
                    'P(-1) not quant match':
                        not bool(quant['match']),
                    
                    'P(-1) != adjv[match][target]':
                        P(-1) != adjv['match'].get('target', None),
                    
                }
                
            },
            
            {
                'name': 'conjunction w/ conj accent',
                'src': P(0),
                'tgt': P(-2),
                'cnd':{
                    
                    'P(-1,sp) == conj':
                        P(-1,'sp') == 'conj',
                    
                    'P(-2) in noms':
                        P(-2) in noms,
                    
                    'P(-2) atype == conj':
                        atype.get(P(-2)) == 'conjunct',
                    
                    
                }
                
            },
            
            {
                'name': 'conjunction w/ conj accent (article)',
                'src': P(0),
                'tgt': P(-3),
                'cnd':{
                    
                    'P(-1,sp) == art':
                        P(-1,'sp') == 'art',
                    
                    'P(-2,sp) == conj':
                        P(-2,'sp') == 'conj',
                    
                    'P(-2) in noms':
                        P(-3) in noms,
                    
                    'P(-2) atype == conj':
                        atype.get(P(-3)) == 'conjunct',
                    
                }
                
            },
            
            {
                'name': 'conjunction w/ conj accent (prep)',
                'src': P(0),
                'tgt': P(-3),
                'cnd':{
                    
                    'P(-1,sp) in preps':
                        P(-1) in preps,
                    
                    'P(-2,sp) == conj':
                        P(-2,'sp') == 'conj',
                    
                    'P(-3) in noms':
                        P(-3) in noms,
                    
                    'P(-3) atype == conj':
                        atype.get(P(-3)) == 'conjunct',
                    
                    'P(-4) in preps':
                        P(-4) in preps,
                    
                }
                
            },
            
            {
                'name': 'conjunction w/ conj accent (prep & article)',
                'src': P(0),
                'tgt': P(-4),
                'cnd':{
                    
                    'P(-1,sp) == art':
                        P(-1,'sp') == 'art',
                    
                    'P(-2) in preps':
                        P(-2) in preps,
                    
                    'P(-3,sp) == conj':
                        P(-3,'sp') == 'conj',
                    
                    'P(-4) in noms':
                        P(-4) in noms,
                    
                    'P(-4) atype == conj':
                        atype.get(P(-4)) == 'conjunct',
                    
                    'P(-5,sp) == art':
                        P(-5,'sp') == 'art',
                    
                    'P(-6) in preps':
                        P(-6) in preps,
                    
                }
                
            },
        )
        
        params['match'] = self.test(params['conds'])
        return params